In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
train=pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
test=pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
x=train.iloc[:,1:101]
y=train.loss

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size = .4,random_state =42)

In [ ]:
x_test=test.iloc[:,1:]

In [ ]:
x_test.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm

In [ ]:
modlr=LinearRegression()
modlr=modlr.fit(x,y)
rsq=modlr.score(x,y)
rsq

In [ ]:
y_test_lr=modlr.predict(x_test)
y_test_lr

In [ ]:
modrt=tree.DecisionTreeRegressor(min_samples_split=10000)
modrt=modrt.fit(x_train,y_train)
paramgrid=[{"min_samples_split":[10000,20000]}]
search=GridSearchCV(modrt,paramgrid,scoring="neg_mean_squared_error",cv=3,verbose=True,n_jobs=-1)
search=search.fit(x_train,y_train)

In [ ]:
search.best_params_

In [ ]:
modrt=tree.DecisionTreeRegressor(min_samples_split=20000)
modrt=modrt.fit(x_train,y_train)
rsq_val=modrt.score(x_val,y_val)
rsq

In [ ]:
y_test_tree=modrt.predict(x_test)
y_test_tree

In [ ]:
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

In [ ]:
alphas = 10**np.linspace(10,-2,100)*0.5
alphas

In [ ]:
ridge = Ridge(normalize = True)
coefs = []

for a in alphas:
    ridge.set_params(alpha = a)
    ridge.fit(x, y)
    coefs.append(ridge.coef_)
    
np.shape(coefs)

In [ ]:
ridge2 = Ridge(alpha = 4, normalize = True)
ridge2.fit(x_train, y_train)             # Fit a ridge regression on the training data
pred2 = ridge2.predict(x_val)           # Use this model to predict the test data
print(pd.Series(ridge2.coef_, index = x.columns)) # Print coefficients
print(mean_squared_error(y_val, pred2))          # Calculate the test MSE

In [ ]:
ridge3 = Ridge(alpha = 10**10, normalize = True)
ridge3.fit(x_train, y_train)             # Fit a ridge regression on the training data
pred3 = ridge3.predict(x_val)           # Use this model to predict the test data
print(pd.Series(ridge3.coef_, index = x.columns)) # Print coefficients
print(mean_squared_error(y_val, pred3))          # Calculate the test MSE

In [ ]:
ridge2 = Ridge(alpha = 0, normalize = True)
ridge2.fit(x_train, y_train)             # Fit a ridge regression on the training data
pred = ridge2.predict(x_val)            # Use this model to predict the test data
print(pd.Series(ridge2.coef_, index = x.columns)) # Print coefficients
print(mean_squared_error(y_val, pred))           # Calculate the test MSE

In [ ]:
ridgecv = RidgeCV(alphas = alphas, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(x_train, y_train)
ridgecv.alpha_

In [ ]:
ridge4 = Ridge(alpha = ridgecv.alpha_, normalize = True)
ridge4.fit(x_train, y_train)
mean_squared_error(y_val, ridge4.predict(x_val))

In [ ]:
ridge4.fit(x, y)
pd.Series(ridge4.coef_, index = x.columns)

In [ ]:
y_test_ridge = ridge4.predict(x_test)
y_test_ridge

In [ ]:
lasso = Lasso(max_iter = 10000, normalize = True)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(scale(x_train), y_train)
    coefs.append(lasso.coef_)

In [ ]:
lassocv = LassoCV(alphas = None, cv = 20, max_iter = 100000, normalize = True)
lassocv.fit(x_train, y_train)

lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(x_train, y_train)
mean_squared_error(y_val, lasso.predict(x_val))

In [ ]:
pd.Series(lasso.coef_, index=x.columns)

In [ ]:
y_test_lasso = lasso.predict(x_test)
y_test_lasso

In [ ]:
from sklearn import model_selection
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression, PLSSVD

In [ ]:
pca = PCA()
X_reduced = pca.fit_transform(scale(x))

In [ ]:
pd.DataFrame(pca.components_.T).loc[:4,:5]

In [ ]:
n = len(X_reduced)
kf_10 = model_selection.KFold( n_splits=10, shuffle=True, random_state=1)

regr = LinearRegression()
mse = []

# Calculate MSE with only the intercept (no principal components in regression)
score = -1*model_selection.cross_val_score(regr, np.ones((n,1)), y.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()    
mse.append(score)

# Calculate MSE using CV for the 19 principle components, adding one component at the time.
for i in np.arange(1, 20):
    score = -1*model_selection.cross_val_score(regr, X_reduced[:,:i], y.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(score)
    

In [ ]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

In [ ]:
pca2 = PCA()
# Scale the data
X_reduced_train = pca2.fit_transform(scale(x_train))
n = len(X_reduced_train)

# 10-fold CV, with shuffle
kf_10 = model_selection.KFold( n_splits=10, shuffle=True, random_state=1)

mse = []

# Calculate MSE with only the intercept (no principal components in regression)
score = -1*model_selection.cross_val_score(regr, np.ones((n,1)), y_train.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()    
mse.append(score)

# Calculate MSE using CV for the 19 principle components, adding one component at the time.
for i in np.arange(1, 20):
    score = -1*model_selection.cross_val_score(regr, X_reduced_train[:,:i], y_train.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(score)

In [ ]:
X_reduced_val = pca2.transform(scale(x_val))[:,:7]

# Train regression model on training data 
regr = LinearRegression()
regr.fit(X_reduced_train[:,:7], y_train)

# Prediction with test data
pred = regr.predict(X_reduced_val)
mean_squared_error(y_val, pred)

In [ ]:
X_reduced_test = pca2.transform(scale(x_test))[:,:7]
y_test_pca = regr.predict(X_reduced_test)
y_test_pca

In [ ]:
n = len(x_train)

# 10-fold CV, with shuffle
kf_10 = model_selection.KFold(n_splits=10, shuffle=True, random_state=1)

mse = []

for i in np.arange(1, 20):
    pls = PLSRegression(n_components=i)
    score = model_selection.cross_val_score(pls, scale(x_train), y_train, cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(-score)

In [ ]:
import matplotlib.pyplot as plt
# Plot results
plt.plot(np.arange(1, 20), np.array(mse), '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('MSE')
plt.title('Loss')
plt.xlim(xmin=-1)

In [ ]:
pls = PLSRegression(n_components=3)
pls.fit(scale(x_train), y_train)

mean_squared_error(y_val, pls.predict(scale(x_val)))

In [ ]:
y_test_pls = pls.predict(scale(x_test))
y_test_pls

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)
rsq_val=clf.score(x_val,y_val)
rsq

In [ ]:
y_test_gb = clf.predict(x_test)
y_test_gb

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
modknn=KNeighborsRegressor(n_neighbors=5)
modknn=modknn.fit(x_train,y_train)
rsq=modknn.score(x_val,y_val)
rsq

In [ ]:
y_test_knn = modknn.predict(x_test)
y_test_knn

In [ ]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

In [ ]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
        "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
        "nthread" : 4
}

In [ ]:
model = lgb.LGBMRegressor(**params)
m_lgb = model.fit(x_train, y_train,
        eval_set=[(x_val, y_val)],
        eval_metric='rmse',
        early_stopping_rounds=5) 

In [ ]:
y_test_lgb = model.predict(x_test)
y_test_lgb

In [ ]:
model1 = xgb.XGBRegressor()
m_xgb = model1.fit(x_train, y_train,
        eval_set=[(x_val, y_val)],
        eval_metric='rmse',
        early_stopping_rounds=5) 

In [ ]:
y_test_xgb = model1.predict(x_test)
y_test_xgb

In [ ]:
model2=cb.CatBoostRegressor()
m_cb = model2.fit(x_train, y_train,
        eval_set=[(x_val, y_val)],
        early_stopping_rounds=5) 

In [ ]:
y_test_cb = model2.predict(x_test)
y_test_cb

In [ ]:
submission = pd.DataFrame({
    'id': test.id, 
    "loss": y_test_ridge
})
submission.to_csv('t1.csv', index=False)
print(submission)